# Updating Compositions

**Purpose:** Discover if when a `Composition` is updated, whether all resources that have already been created using it get updated automatically.

**Reason:** As a platform, we provide capabilities through APIs defined in XRDs. Our users consume those APIs with composites and claims. We need to deliver updates to the implementations for resources they have already created, without requiring the users to take action.

## Findings
1. When a `Composition` is updated, it is re-applied for all existing composites and claims, and the resources it creates are updated.
2. This behaviour can be prevented by a consumer by adding `compositionUpdatePolicy` to their claim or composite.
3. The consumer can then adopt newer revisions of the `Composition` by setting `compositionRevisionRef` on their claim or composite.
4. Composition revisions are not too useful as a way of allowing consumers to choose their implementation version since the revision names will not be stable across clusters.

## Setup

Let's define an XRD for a simple resource with a single property, `foo`:

In [2]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: CompositeResourceDefinition
metadata:
  name: xsimples.example.guidewire.net
spec:
  group: example.guidewire.net
  names:
    kind: xSimple
    plural: xsimples
  claimNames:
    kind: Simple
    plural: simples
  versions:
  - name: v1alpha1
    served: true
    referenceable: true
    schema:
      openAPIV3Schema:
        type: object
        properties:
          spec:
            type: object
            properties:
              foo:
                type: string
EOF

compositeresourcedefinition.apiextensions.crossplane.io/xsimples.example.guidewire.net created


Let's also define an implementation that creates a `ConfigMap` with the value of `foo` put in it:

In [3]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition
spec:
  mode: Pipeline
  compositeTypeRef:
    apiVersion: example.guidewire.net/v1alpha1
    kind: xSimple
  pipeline:
  - step: patch-and-transform
    functionRef:
      name: function-patch-and-transform
    input:
      apiVersion: pt.fn.crossplane.io/v1beta1
      kind: Resources
      resources:
      - name: themap
        base:
          apiVersion: kubernetes.crossplane.io/v1alpha2
          kind: Object
          spec:
            forProvider:
              manifest:
                apiVersion: v1
                kind: ConfigMap
                metadata:
                  namespace: default
                data:
                  constant: first version
                  foo: replace me
        patches:
          - type: FromCompositeFieldPath
            fromFieldPath: spec.foo
            toFieldPath: spec.forProvider.manifest.data.foo
EOF

composition.apiextensions.crossplane.io/simple-composition created


Now we can create a claim and see that the `ConfigMap` gets created:

In [4]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test
  namespace: default
spec:
  foo: "Hello, World!"
  compositionRef:
    name: simple-composition 
EOF

simple.example.guidewire.net/test created


In [19]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        7m41s
-----------------
Name:         test-l8vxb-vcblj
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
first version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Note that a composite was also created by the claim:

In [21]:
%%bash
kubectl get composite `kubectl get claim test -o yaml | yq .spec.resourceRef.name`

NAME         SYNCED   READY   COMPOSITION          AGE
test-l8vxb   True     True    simple-composition   8m29s


## Experiment: Update the `Composition`

The `Composition` created a `ConfigMap` with two keys: `foo` and `constant`. The value of `foo` is provided through the XRD; the value of `constant` is provided in the `Composition` itself.

**Hypothesis:** if we update the `Composition` with a new value for `constant`, then the existing `ConfigMap` should be updated in place with the new value of `constant`.

Let's update the value of `constant` in the `Composition`:

In [22]:
%%bash
cat << EOF | kubectl apply -f -
apiVersion: apiextensions.crossplane.io/v1
kind: Composition
metadata:
  name: simple-composition
spec:
  mode: Pipeline
  compositeTypeRef:
    apiVersion: example.guidewire.net/v1alpha1
    kind: xSimple
  pipeline:
  - step: patch-and-transform
    functionRef:
      name: function-patch-and-transform
    input:
      apiVersion: pt.fn.crossplane.io/v1beta1
      kind: Resources
      resources:
      - name: themap
        base:
          apiVersion: kubernetes.crossplane.io/v1alpha2
          kind: Object
          spec:
            forProvider:
              manifest:
                apiVersion: v1
                kind: ConfigMap
                metadata:
                  namespace: default
                data:
                  constant: second version
                  foo: replace me
        patches:
          - type: FromCompositeFieldPath
            fromFieldPath: spec.foo
            toFieldPath: spec.forProvider.manifest.data.foo

composition.apiextensions.crossplane.io/simple-composition configured


Now let's check if the `ConfigMap` was updated:

In [23]:
%%bash
kubectl describe `kubectl get configmap -o=name | grep test`

Name:         test-l8vxb-vcblj
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
second version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


The `constant` key's value has been updated from `first version` to `second version`.

Note that the name of the `ConfigMap` is the same as before, `test-l8vxb-vcblj`, which confirms it was updated in-place.

This confirms the hypothesis that updates to the `Composition` will be applied to existing resources.

## Experiment: Locking a Claim to a `Composition`

**Hypothesis:** A consumer can use the `compositionUpdatePolicy` on a composite or claim to prevent updates from the `Composition` from being applied.

Delete and re-create the claim with `compositionUpdatePolicy` set to `Manual`:

In [24]:
%%bash
kubectl delete claim test
cat << EOF | kubectl apply -f -
apiVersion: example.guidewire.net/v1alpha1
kind: Simple
metadata:
  name: test
  namespace: default
spec:
  foo: "Hello, World!"
  compositionRef:
    name: simple-composition
  compositionUpdatePolicy: Manual
EOF

simple.example.guidewire.net "test" deleted
simple.example.guidewire.net/test created


Let's view the `ConfigMap`:

In [25]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        45s
-----------------
Name:         test-nq9gn-4csft
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
second version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


Update the `Composition` again by changing the hard-coded value it uses for `constant`:

In [26]:
%%bash
kubectl patch composition simple-composition --type='json' -p='[{"op": "replace", "path": "/spec/pipeline/0/input/resources/0/base/spec/forProvider/manifest/data/constant", "value":"third version"}]'

composition.apiextensions.crossplane.io/simple-composition patched


View the `ConfigMap` again:

In [27]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        5m30s
-----------------
Name:         test-nq9gn-4csft
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
foo:
----
Hello, World!
constant:
----
second version

BinaryData
====

Events:  <none>


As expected, it is not updated.

We can make the claim use the new revision of the `Composition` as follows:

In [48]:
%%bash
kubectl get compositionrevision
echo "-----------------"
kubectl get claim test
echo "-----------------"
kubectl patch claim test -p="{\"spec\":{\"compositionRevisionRef\":{\"name\":\"$(kubectl get compositionrevision | grep "simple-composition" | sort -k 2 | tail -1 | cut -f1 -d' ')\"}}}" --type=merge

NAME                         REVISION   XR-KIND   XR-APIVERSION                    AGE
simple-composition-0b06493   3          xSimple   example.guidewire.net/v1alpha1   10m
simple-composition-6a700d0   1          xSimple   example.guidewire.net/v1alpha1   129m
simple-composition-79696e3   2          xSimple   example.guidewire.net/v1alpha1   109m
-----------------
NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        15m
-----------------
simple.example.guidewire.net/test patched


View the `ConfigMap` again:

In [49]:
%%bash
kubectl get claim test
echo "-----------------"
kubectl describe `kubectl get configmap -o=name | grep test`

NAME   SYNCED   READY   CONNECTION-SECRET   AGE
test   True     True                        15m
-----------------
Name:         test-nq9gn-4csft
Namespace:    default
Labels:       <none>
Annotations:  <none>

Data
====
constant:
----
third version
foo:
----
Hello, World!

BinaryData
====

Events:  <none>


The `constant` key's value has been updated from `second version` to `third version`.

This shows that a `compositionRevisionRef` can be used on a claim to use the latest `Composition` in cases where `compositionUpdatePolicy` is set to `Manual`.